In [ ]:
!git clone https://github.com/ankanbhunia/Handwriting-Transformers
%cd Handwriting-Transformers
!pip install --upgrade --no-cache-dir gdown
!gdown --id 16g9zgysQnWk7-353_tMig92KsZsrcM6k && unzip files.zip && rm files.zip

In [24]:
import os
import time
from data.dataset import load_itw_samples
import torch
import cv2
import os
import numpy as np
from models.model import TRGAN
from params import *
from torch import nn
from data.dataset import get_transform
import pickle
from PIL import Image
import tqdm
import shutil

In [25]:
text = "Welcome to CS787 Generative AI course. This is a sample text to test the Handwriting Transformer model. This model is a GAN based model that has 4 components Generator, Discriminator, Recognizer, Writer identifier."
output_path = 'results'

In [26]:
model_path = 'saved_models/IAM-339-15-E3D3-LR0.00005-bs8/model_final.pth'
#model_path = 'files/cvl_model.pth'


In [27]:
# --- Robust checkpoint loader (add this once near your imports) ---
import torch
from collections import OrderedDict

def _clean_keys(sd, prefix_drop_list=("module.",), replace_prefixes=()):
    new_sd = OrderedDict()
    for k, v in sd.items():
        kk = k
        for p in prefix_drop_list:
            if kk.startswith(p):
                kk = kk[len(p):]
        for oldp, newp in replace_prefixes:
            if kk.startswith(oldp):
                kk = newp + kk[len(oldp):]
        new_sd[kk] = v
    return new_sd

def load_legacy_trgan(model, model_path, load_netG_only=True, device="cuda:0"):
    """
    Load legacy checkpoints into the new TRGAN/Generator.
    - load_netG_only=True: load only Generator weights (works for inference)
    - strict=False so new layers (curve_* etc.) stay randomly init’d
    Returns (missing_keys, unexpected_keys) from load_state_dict.
    """
    ckpt = torch.load(model_path, map_location="cpu")

    # Accept common formats
    if isinstance(ckpt, dict) and "state_dict" in ckpt:
        sd = ckpt["state_dict"]
    elif isinstance(ckpt, dict) and any(k in ckpt for k in ("netG", "netD", "netOCR", "netW")):
        sd = ckpt.get("netG", ckpt) if load_netG_only else ckpt
    else:
        sd = ckpt  # raw state_dict

    # If wrapper dict but asked only for netG
    if load_netG_only and isinstance(sd, dict) and not any(k.endswith(".weight") or k.endswith(".bias") for k in sd.keys()):
        sd = sd.get("netG", sd)

    # Clean DP/nested prefixes
    sd = _clean_keys(
        sd,
        prefix_drop_list=("module.",),
        replace_prefixes=(("netG.module.", ""), ("netG.", ""))
    )

    missing, unexpected = [], []
    if load_netG_only:
        tgt = model.netG.state_dict()
        filtered = {k: v for k, v in sd.items() if k in tgt and v.shape == tgt[k].shape}
        info = model.netG.load_state_dict(filtered, strict=False)
        missing += list(info.missing_keys)
        unexpected += list(info.unexpected_keys)
    else:
        tgt = model.state_dict()
        filtered = {k: v for k, v in sd.items() if k in tgt and v.shape == tgt[k].shape}
        info = model.load_state_dict(filtered, strict=False)
        missing += list(info.missing_keys)
        unexpected += list(info.unexpected_keys)

    model.to(device).eval()
    return missing, unexpected


In [28]:
# batch_size = 1
# print ('(1) Loading model...')
# model = TRGAN(batch_size = batch_size)
# model.load_state_dict(torch.load(model_path))
# print (model_path+' : Model loaded Successfully')
# model.eval()
# print ('(2) Loading text content...')
# text_encode =  [j.encode() for j in text.split(' ')]
# eval_text_encode, eval_len_text = model.netconverter.encode(text_encode)
# eval_text_encode = eval_text_encode.to('cuda:0').repeat(batch_size, 1, 1)
# print ('Done.')


In [29]:
batch_size = 1
device = "cuda:0" if torch.cuda.is_available() else "cpu"

print('(1) Loading model...')
model = TRGAN(batch_size=batch_size).to(device)

# Use robust loader (handles old checkpoints / DP prefixes / netG-only)
missing, unexpected = load_legacy_trgan(
    model,
    model_path,
    load_netG_only=True,   # set False only if your ckpt is a full model state_dict
    device=device
)
print(f"{model_path} : Model loaded (non-strict).")
if missing:
    print("[Info] Missing keys (expected for new layers):", missing[:10], "..." if len(missing) > 10 else "")
if unexpected:
    print("[Info] Unexpected keys (ignored):", unexpected[:10], "..." if len(unexpected) > 10 else "")

print('(2) Loading text content...')
text_bytes = [j.encode() for j in text.split(' ')]
eval_text_encode, eval_len_text = model.netconverter.encode(text_bytes)
eval_text_encode = eval_text_encode.to(device).repeat(batch_size, 1, 1)
print('Done.')


(1) Loading model...
initialize network with N02
initialize network with N02
initialize network with N02


C:\Users\jayag\AppData\Local\Temp\ipykernel_14748\1569221876.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path, map_location="cpu")


saved_models/IAM-339-15-E3D3-LR0.00005-bs8/model_final.pth : Model loaded (non-strict).
[Info] Missing keys (expected for new layers): ['encoder.layers.0.self_attn.in_proj_weight', 'encoder.layers.0.self_attn.in_proj_bias', 'encoder.layers.0.self_attn.out_proj.weight', 'encoder.layers.0.self_attn.out_proj.bias', 'encoder.layers.0.linear1.weight', 'encoder.layers.0.linear1.bias', 'encoder.layers.0.linear2.weight', 'encoder.layers.0.linear2.bias', 'encoder.layers.0.norm1.weight', 'encoder.layers.0.norm1.bias'] ...
(2) Loading text content...
Done.


In [30]:
## load your *cropped word images* of the custom handwriting here. Images do NOT need to be scaled, resized or padded. load_itw_samples(.) will take care of them.

style_inputs, width_length = load_itw_samples('custom_data_1/style_examples/')

page_val = model._generate_page(style_inputs.to(DEVICE).clone(), width_length, eval_text_encode, eval_len_text)

cv2.imwrite('output_new.png', page_val*255)

True